In [1]:
%load_ext autoreload

%autoreload 2

In [18]:
import main_agent as ma
import prompts_lib as pl


In [12]:
message = pl.system_prompt2 + 'what is the capital of france'

In [13]:
response = ma.graph.invoke({'messages': message})

In [14]:
response.get('messages')[1].content

'Paris'

In [28]:
class BasicAgent:
    def __init__(self):
        pass
    def __call__(self, question: str) -> str:
        print(f"Agent received question (first 50 chars): {question[:50]}...")

        message = pl.system_prompt2 + question

        response = ma.agent.invoke({'messages': message})

        answer = response.get('messages')[1].content

        # fixed_answer = "This is a default answer."
        print(f"Agent returning answer: {answer}")
        return answer

In [29]:
agent = BasicAgent()

In [30]:
answer = agent("What is the capital of Italy?")

Agent received question (first 50 chars): What is the capital of Italy?...
Agent returning answer: Rome
